## dp实施原则

三个原则实施DP:
```
1. 可划分阶段,(理解就是可划分子问题,可递推)
2. 最优化原理,即子问题的局部最优将导致整个问题的全局最优，即问题具有最优子结构的性质,
   也就是说一个问题的最优解只取决于其子问题的最优解
3. 无后效性,即当前的状态只和过去有关,和未来无关,有点像线性时不变里的因果性
```

动态规划与分治法类似，都是把大问题拆分成小问题，通过寻找大问题与小问题的递推关系，解决一个个小问题，最终达到解决原问题的效果。但不同的是，分治法在子问题和子子问题等上被重复计算了很多次，而动态规划则具有记忆性，通过填写表把所有已经解决的子问题答案纪录下来，在新问题里需要用到的子问题可以直接提取，避免了重复计算，从而节约了时间，所以在问题满足最优性原理之后，用动态规划解决问题的核心就在于填表，表填写完毕，最优解也就找到。

## 模型

上升,下降,或者其他场景都应该借鉴,举一反三

### 最长上升子序列的长度(不一定连续)


```
最长上升子序列可能以任何位置作为结尾! 这就是`划分子问题`(方式并不唯一!), 即: dp[n]是以a[n]结尾的最长子序列的长度;

dp[n]可以递推, 其值可能和dp[0],dp[1],...dp[n-1]有关, 这是`无后效性`

所有的dp[n]求得时,全局问题自然得解,即max(dp)
```

最重要的就是找出dp[i]的定义,和递推关系

dp[i]表示以a[i]结尾的上升子序列的长度, max(dp)即求解问题

```
step: 0 , dp[0] = 1
step: i, 对于j in range(1,i) a[j]是dp[j]代表的最长子序列的结尾, if a[i] > a[j],此时可以形成新的最长子序列 长度为a[j]+1,
    找到所有dp[j]中,可能的最大的dp[j]+1, 用来更新dp[i]
```

In [1]:
class Solution:
    def maxSubArrayLength(self,nums):
        dp = []
        [dp.append(1) for i in range(0,len(nums))]
        print(dp)
        for i in range(1,len(nums)):
            for j in range(1,i):
                dp[i] = max(dp[i],dp[j]+1) if nums[i] > nums[j] else dp[i]
        print(dp)
        return max(dp)

### 连续子序列的最大和

ltcode里`53. Maximum Subarray`



这个要更简单点,因为不考虑上升性:,
    dp[i] 表示以 a[i]结尾的子序列的和
```
step 0: dp[0] = 1
step i: dp[i] = dp[i-1] + num[i] if  dp[i-1] > 0 else num[i]
```
为何要dp[i-1] > 0 ? 如果dp[i]小于0,说明前面序列累加起来再加上现在的,比现在的还小,最长子序列不成立了.

In [5]:
class Solution:
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        dp = nums.copy()
        for i in range(1, len(nums)):
            if dp[i-1] > 0:
                dp[i] = dp[i-1] + nums[i]
        return max(dp)

### 连续上升子序列的最大和

和上面的比,仅仅增加是否连续的判断

```
dp[i] = dp[i-1] + nums[i] if nums[i] < nums[i-1] and dp[i-1] > 0 else nums[i]
```

In [3]:
class Solution:
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        dp = []
        l = len(nums)
        dp = nums.copy()
        dp[0] = nums[0]
        for i in range(1,l):
            dp[i] = dp[i-1] + nums[i] if nums[i] < nums[i-1] and dp[i-1] > 0 else nums[i]
        return max(dp)

## 应用

### 合唱团问题

N个成员的合唱团,身高为a[i], 希望拿掉(N-K)个成员,剩余成员构成'合唱团',即
```
a1 < a2 < .. ai
ai > ai+1 >...aK
```
求K的最大值?

分析: 是2个子问题(上面2.1)的结合:
```
dp[i] 表示以a[i]为结尾的递增子序列(不用连续)的队列长度
dq[i] 表示以a[i]为开头的递减子序列(不用连续)的队列长度

求解问题为max(d[i]+dq[i]-1) 减1是因为i重复算了1次

```

### 背包问题

问题:
```
有n 个物品，它们有各自的重量和价值，现有给定容量的背包，如何让背包里装入的物品具有最大的价值总和？
```

#### 胡思乱想1

分析:
```
很类似最大子序列的问题 
容量和不超过V, 价值和最大
```

ps:完全自己的思路...没参考过任何人

dp[i] 表示放入第i个物品后, 已经用掉的体积. 这里存在1个顺序放的概念

怎么规定顺序呢?  让物品体积和价值做1个权重 比如 价值/体积 ,再排序


```
dp[0] = vol[0] if V - vol[0] >= 0 else -1
dp[i] = dp[i-1] + vol[i] if V - dp[i-1] > vol[i] and dp[i-1] != -1 else -1 

```

最后的价值应该为不为-1的那个dp[i] 之前的所有序号 的val加起来 即使问题求解

后面证明, 这个权重是有问题的...
[2:4 4:8 1:1 1:1] 需要6个

    

In [24]:
a=[1,2,3,-1,-1]

In [25]:
a.index(-1)

3

In [55]:
class Solution:
    def maxValues(self,vol,val,V):
        seq = [ va/vo for va,vo in  zip(val,vol)]
        # 不同体积 价值,算出的比例一致怎么办? 应该取较小的值
        dicts = {}
        for i,s in enumerate(seq):
            if s not in dicts:
                dicts[s] = i
            else:
                dicts[s] = min(i, dicts[s])
        
        print('权重字典: 权重:原序号 相同权重时,取比较小的体积 万一正好就这样2个体积')
        
        print(dicts)
        
        sort_seq = sorted(dicts,reverse=True)
        
        sort_index = [ dicts[i] for i in sort_seq ]
        
        print('权重序列 排序: ')
        print(sort_index)
        
        
        dp  = []
        
        [dp.append(-1) for i in range(0,len(seq)) ]
        
        print('初始dp: ')
        print(dp)
        
      
        
        dp[0] = vol[ sort_index[0] ] if V - vol[ sort_index[0] ] >= 0 else -1
        
        def find_maxvalues_for_v(indexs,start,end,vol,val,dp,left_V):
            pass
        
        for i in range(1,len(sort_index)):
            dp[i] = dp[i-1] + vol[sort_index[i]] \
            if V - dp[i-1] >= vol[sort_index[i]] and dp[i-1] != -1 else -1
        
        print('推演后的dp. -1表示体积满了,不能再塞了 ')
        print(dp)
        
        try:
            end = dp.index(-1)
        except Exception as e:
            print('没有-1全部都可以 容下')
            return sum(val)
        

        print('dp结束的位置')
        print(end)
        print(sort_index[:end])
        print('先算体积是不是满了?没满还能不能塞其他的?')
        used_vols_lists = [ vol[i] for i in sort_index[:end] ]
        print(used_vols_lists)
        vals = sum([ val[i] for i in sort_index[:end] ])
        used_vols = sum(used_vols_lists)
        left_vols = V-used_vols
        
        for i in sort_index[end:]:
            if vol[i] == left_vols:
                vals += val[i]
                
        return vals
        
        

In [56]:
Solution().maxValues([1,2,3,1,2,3,4,5,1],[4,5,7,1,0,2,3,6,3], 3)

权重字典: 权重:原序号 相同权重时,取比较小的体积
{4.0: 0, 2.5: 1, 2.3333333333333335: 2, 1.0: 3, 0.0: 4, 0.6666666666666666: 5, 0.75: 6, 1.2: 7, 3.0: 8}
权重序列 排序: 
[0, 8, 1, 2, 7, 3, 6, 5, 4]
初始dp: 
[-1, -1, -1, -1, -1, -1, -1, -1, -1]
推演后的dp. -1表示体积满了,不能再塞了 
[1, 2, -1, -1, -1, -1, -1, -1, -1]
dp结束的位置
2
[0, 8]
先算体积是不是满了?没满还能不能塞其他的?
[1, 1]


8

体积满 了和没满 不是一个等级! 最后应该再看看还能不能再塞1个

体积3
```
vol 1, 2 = val 4 + 5
vol 1, 1, 1 = val 4 + 3 + 1?
```

但是8还是比9小!...思路有点问题 

```
剩余空间一定分配给下1个权重最高的序列? 如果造成了体积剩余 剩余==效率低下!

if vol[i] == V - dp[i-1] : dp[i] = dp[i-1] + vol[i] 很好, 全部用完,完美
elif vol[i] > V - dp[i-1] : 放不下了 dp[i] = -1
elif vol[i] < V - dp[i-1] : 放下去还剩的有呢? 


如果判断是否有浪费? 即当V-dp[i-1]-vol[i] 能被1次效率很高的塞满,(多次不行吗?) 

假设 剩余的体积能够在剩余的权重序列里,能拿到的最大效率Vals...递归了???....

这就是贪心法的不足!...

```

#### 正规解法

```
定义子问题 {P(i, W) 为：在前i个物品中挑选总重量不超过 W 的物品，每种物品至多只能挑选1个，使得总价值最大；这时的最优值记作 m(i,w) ，其中 1<=i<=n, 1<=w<=W , w表示背包还剩余的容量
```

对于第i个物品, 选or 不选?
```
i太大, 不选:   m(i,w) = m(i-1,w) 
选,容量减少,价值可能增加     m(i,w) = m(i-1,w-weight[i]) + val[i]

总结:
    
m(i,w) = 

i = 0, m[0,w] = 0
w = 0, m[i,w] = 0
对所有的i,w:
    if weight[i] >=  w:  m(i,w) = m(i-1,w):
    else: 
        m(i,w) = max(  m(i-1,w) ,m(i-1,w-weight[i]) + val[i] )
```

https://zhuanlan.zhihu.com/p/30959069

In [245]:
class Solution:
    def bagProblem(self,weights,values,capacity):
        l = len(weights)
        dp = []
        # i 表示第i个物品,需要补全一下
        weights.insert(0,0)
        values.insert(0,0)
        for i in range(0,l):
            dp.append([])
            for c in range(0,capacity+1):
                dp[i].append(0)
                
#         print(dp)
        # dp[i,0] or dp[0,i] = 0
       
        for i in range(1,l):
            for j in range(1,capacity+1):
                if j < weights[i]:
                    dp[i][j]  = dp[i-1][j]
                else:
                    dp[i][j] = max( dp[i-1][j-weights[i]] + values[i], dp[i-1][j])
        print(dp)
        return dp[i][j]

In [246]:
weights = [1,2,5,6,7]
values =  [1,6,18,22,28]
Solution().bagProblem(weights,values,11)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7], [0, 1, 6, 7, 7, 18, 19, 24, 25, 25, 25, 25], [0, 1, 6, 7, 7, 18, 22, 24, 28, 29, 29, 40]]


40

#### cpu问题

问题的变形:(2017 Netease)
```
一种双核CPU的两个核能够同时的处理任务，现在有n个已知数据量的任务需要交给CPU处理，假设已知CPU的每个核1秒可以处理1kb，每个核同时只能处理一项任务。n个任务可以按照任意顺序放入CPU进行处理，现在需要设计一个方案让CPU处理完这批任务所需的时间最少，求这个最小的时间。

输入描述: 输入包括两行： 第一行为整数n(1 ≤ n ≤ 50) ，第二行为n个整数length[i](1024 ≤ length[i] ≤ 4194304)，表示每个任务的长度为length[i]kb，每个数均为1024的倍数。输出描述: 输出一个整数，表示最少需要处理的时间输入例子1: 5 3072 3072 7168 3072 1024输出例子1: 9216

```

分析:
```
2个cpu拿到的任务处理时间之差尽量小.  即对于sum1/sum2来说,尽量接近SUM/2 , 转换为简单的背包问题:

速度一致才可以计算SUM,而且讲计算全部转换为时间即 计算量/速度. 

sum1+sum2 = SUM

sum1-SUM/2 = SUM/2 - sum2 

V = SUM / 2 (类似背包问题的总容量限制)

dp[i][j] 分配到前i个任务时,时间的容量为j ,完全和背包问题一样了

```
 

In [105]:
[i for i in range(9,1-1,-1)]

[9, 8, 7, 6, 5, 4, 3, 2, 1]

In [258]:
class Solution:
    def cpuProblem(self,nums,capacity):
        dp = []
        nums.insert(0,0)
        l = len(nums)
        for i in range(0,l):
            dp.append([])
            for c in range(0,capacity+1):
                dp[i].append(0)
                
#         print(dp)
        # dp[i,0] or dp[0,i] = 0
        # 注意,利用了: dp[-1] = 0 ,坐标溢出
        for i in range(1,l):
            for j in range(1,capacity+1):
                if j < nums[i]:
                    dp[i][j]  = dp[i-1][j]
                else:
                    dp[i][j] = max( dp[i-1][j-nums[i]] + nums[i], dp[i-1][j])
        print(dp)
        return dp[i][j]

In [259]:
tasks = [1,2,3,4,5]

Solution().cpuProblem(tasks, 11)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3], [0, 1, 2, 3, 4, 5, 6, 6, 6, 6, 6, 6], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]


11

## 参考

1. https://blog.csdn.net/cr496352127/article/details/77934132?locationNum=10&fps=1